In [1]:
from pyspark.sql import SparkSession

In [2]:
spark_session = SparkSession.builder.appName("my-test").getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/11/29 18:39:05 WARN Utils: Your hostname, lang-chain-ASUS-TUF-Gaming-A15-FA506QM-FA506QM, resolves to a loopback address: 127.0.1.1; using 192.168.1.86 instead (on interface wlp3s0)
25/11/29 18:39:05 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
data = [(1, "Alice", 25, "Engineering"), 
        (2, "Bob", 30, "Marketing"),
        (3, "Charlie", 35, "Engineering"),
        (4,"john",40,"Math"),
        (5,"harry",30,"Math"),
        (6,"Dow",None,"Math")]
df = spark_session.createDataFrame(data, ["id", "name", "age", "department"])


In [4]:
df.printSchema()

root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- age: long (nullable = true)
 |-- department: string (nullable = true)



In [5]:
df.show()

+---+-------+----+-----------+
| id|   name| age| department|
+---+-------+----+-----------+
|  1|  Alice|  25|Engineering|
|  2|    Bob|  30|  Marketing|
|  3|Charlie|  35|Engineering|
|  4|   john|  40|       Math|
|  5|  harry|  30|       Math|
|  6|    Dow|NULL|       Math|
+---+-------+----+-----------+



In [6]:
from pyspark.sql import functions as F
null_counts = df.select([
    F.sum(F.col(c).isNull().cast("int")).alias(c) 
    for c in df.columns
])

In [7]:
null_counts.show()

+---+----+---+----------+
| id|name|age|department|
+---+----+---+----------+
|  0|   0|  1|         0|
+---+----+---+----------+



In [8]:
mean_age = df.filter(F.col('age').isNotNull()).select(F.avg("age")).first()[0]
mean_age

32.0

In [9]:
avg_age_by_dept = df.groupBy(F.col('department')).agg(
    F.avg('age').alias('avg_age')
)
avg_age_by_dept.show()

+-----------+-------+
| department|avg_age|
+-----------+-------+
|Engineering|   30.0|
|  Marketing|   30.0|
|       Math|   35.0|
+-----------+-------+



In [10]:
dept_age = df.groupBy(F.col("department")).agg(F.avg('age').alias("dep_avg_age"))
dept_age.show()

+-----------+-----------+
| department|dep_avg_age|
+-----------+-----------+
|Engineering|       30.0|
|  Marketing|       30.0|
|       Math|       35.0|
+-----------+-----------+



In [11]:
dept_age_df = df.join(dept_age,on='department',how='left')
dept_age_df.show()

+-----------+---+-------+----+-----------+
| department| id|   name| age|dep_avg_age|
+-----------+---+-------+----+-----------+
|Engineering|  1|  Alice|  25|       30.0|
|  Marketing|  2|    Bob|  30|       30.0|
|Engineering|  3|Charlie|  35|       30.0|
|       Math|  4|   john|  40|       35.0|
|       Math|  5|  harry|  30|       35.0|
|       Math|  6|    Dow|NULL|       35.0|
+-----------+---+-------+----+-----------+



In [12]:
none_null = dept_age_df.withColumn('age',
                    F.coalesce(F.col('age'),
                    F.col('dep_avg_age'))).drop('dep_avg_age')
none_null.show()


+-----------+---+-------+----+
| department| id|   name| age|
+-----------+---+-------+----+
|Engineering|  1|  Alice|25.0|
|  Marketing|  2|    Bob|30.0|
|Engineering|  3|Charlie|35.0|
|       Math|  4|   john|40.0|
|       Math|  5|  harry|30.0|
|       Math|  6|    Dow|35.0|
+-----------+---+-------+----+



In [13]:
from pyspark.sql.window import Window
from pyspark.sql.functions import coalesce
#create specification of department using window partitionBy
department_spec  =Window.partitionBy('department')
#
data = df.withColumn('age',
              coalesce(F.col('age'),
              F.avg('age').over(department_spec)
              )
            )

data.show()

+---+-------+----+-----------+
| id|   name| age| department|
+---+-------+----+-----------+
|  1|  Alice|25.0|Engineering|
|  3|Charlie|35.0|Engineering|
|  2|    Bob|30.0|  Marketing|
|  4|   john|40.0|       Math|
|  5|  harry|30.0|       Math|
|  6|    Dow|35.0|       Math|
+---+-------+----+-----------+



In [14]:
# Calculate overall average age
overall_avg = df.select(F.avg('age')).first()[0]

# Fill nulls with overall average
df_filled = df.withColumn(
    'age',
    F.coalesce(
        F.col('age'),
        F.lit(overall_avg)  # Use overall average as fallback
    )
)

In [15]:
df_filled.show()

+---+-------+----+-----------+
| id|   name| age| department|
+---+-------+----+-----------+
|  1|  Alice|25.0|Engineering|
|  2|    Bob|30.0|  Marketing|
|  3|Charlie|35.0|Engineering|
|  4|   john|40.0|       Math|
|  5|  harry|30.0|       Math|
|  6|    Dow|32.0|       Math|
+---+-------+----+-----------+



In [ ]:
from pyspark.sql.window import Window

window_spec = Window.partitionBy('department')

# Calculate overall average
overall_avg = df.select(F.avg('age')).first()[0]

# Fill with dept avg first, then overall avg
df_filled = df.withColumn(
    'age',
    F.coalesce(
        F.col('age'),                      
        F.avg('age').over(window_spec),    
        F.lit(overall_avg)                 
    )
)
df_filled.show()




+---+-------+----+-----------+
| id|   name| age| department|
+---+-------+----+-----------+
|  1|  Alice|25.0|Engineering|
|  3|Charlie|35.0|Engineering|
|  2|    Bob|30.0|  Marketing|
|  4|   john|40.0|       Math|
|  5|  harry|30.0|       Math|
|  6|    Dow|35.0|       Math|
+---+-------+----+-----------+

